In [1]:
import polars as pl
import numpy as np
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
output_file = os.getenv('EPC_OUTPUT_FILE')
all_epc = pl.read_csv(
    output_file,
    try_parse_dates=True
    #dtype={'transaction-type': str, 'aircon-present': str},
    #parse_dates=['lodgement-date', 'lodgement-datetime']
)
# len(all_epc) # 1378713 before any filtering
# remove small properties
all_epc = all_epc.filter(pl.col('floor-area') >= 40) # 40m2 min size
# and remove prior records for same property
dedup = all_epc.sort('lodgement-datetime').unique(subset=['building-reference-number'], keep='last')
# df.sort_values('DATE_CHANGED').drop_duplicates('STATION_ID',keep='last')
len(dedup) # 1213676 (1281342 before filter by floor area)

1217573

In [6]:
len(dedup.filter(pl.col("uprn").is_null())) # 440164

440164

In [12]:
len(dedup.filter(pl.col("postcode").is_null())) # 0
len(dedup.filter(pl.col("posttown").is_null())) # 161
len(dedup.filter(pl.col("posttown").is_null() & pl.col("uprn").is_null())) # 74
len(dedup.filter(pl.col("constituency-label").is_null())) # 2276
len(dedup.filter(pl.col("constituency-label").is_null() & pl.col("uprn").is_null())) # 955
len(dedup.filter(pl.col("local-authority-label").is_null())) # 761
len(dedup.filter(pl.col("local-authority-label").is_null() & pl.col("uprn").is_null())) # 303

303

In [7]:
dedup.filter(pl.col("posttown").is_null())

lmk-key,address1,address2,address3,postcode,building-reference-number,asset-rating,asset-rating-band,property-type,inspection-date,local-authority,constituency,county,lodgement-date,transaction-type,new-build-benchmark,existing-stock-benchmark,building-level,main-heating-fuel,other-fuel-desc,special-energy-uses,renewable-sources,floor-area,standard-emissions,target-emissions,typical-emissions,building-emissions,aircon-present,aircon-kw-rating,estimated-aircon-kw-rating,ac-inspection-commissioned,building-environment,address,local-authority-label,constituency-label,posttown,lodgement-datetime,primary-energy-value,uprn,uprn-source
str,str,str,str,str,i64,i64,str,str,date,str,str,str,date,str,i64,i64,i64,str,str,str,str,i64,f64,f64,f64,f64,str,f64,f64,i64,str,str,str,str,str,datetime[μs],i64,i64,str
"""8df507b64c84d9f52c5d1ebf4c97bd…",null,"""40 High Street""","""Littlehampton""","""BN17 5ED""",10004937631,27,"""B""","""A1/A2 Retail and Financial/Pro…",2023-08-07,"""E07000224""","""E14000576""","""West Sussex""",2023-08-07,"""Mandatory issue (Marketed sale…",24,70,5,"""Grid Supplied Electricity""",null,null,null,80,77.38,36.88,108.08,42.1,"""No""",null,null,4,"""Air Conditioning""","""40 High Street, Littlehampton""","""Arun""","""Bognor Regis and Littlehampton""",null,2023-08-08 06:24:35,249,10096429526,"""Address Matched"""
"""5e370ca2dbbba04eca65baa40425c3…","""Basement Workshops""","""120 Holborn""",null,"""W1W 6NS""",10003552267,74,"""C""","""Offices and Workshop Businesse…",2022-10-17,"""E09000033""","""E14000639""",null,2022-10-17,"""Mandatory issue (Marketed sale…",0,0,5,"""Grid Supplied Electricity""",null,null,null,463,3.66,0.0,0.0,5.4,"""Yes""",null,2.0,5,"""Air Conditioning""","""Basement Workshops, 120 Holbor…","""Westminster""","""Cities of London and Westminst…",null,2022-10-18 13:22:40,57,null,null
"""a1907e04ecd9343326d35a5d79c358…","""Unit G6""","""Oslo Close""","""Tyne Tunnel Trading Estate""","""NE29 7SZ""",10005969383,43,"""B""","""Storage or Distribution""",2024-04-15,"""E08000022""","""E14001006""",null,2024-04-19,"""Mandatory issue (Property to l…",15,58,3,"""Other""",null,null,null,525,3.87,1.13,4.52,3.33,"""No""",null,null,4,"""Unconditioned""","""Unit G6, Oslo Close, Tyne Tunn…","""North Tyneside""","""Tynemouth""",null,2024-04-19 11:19:24,26,47237196,"""Address Matched"""
"""030a7efb4377993670dc363d29d246…","""Unit B9""","""Whiteley Shopping Centre""","""Whiteley Way""","""PO15 7PD""",10004389480,63,"""C""","""General Assembly and Leisure, …",2022-11-14,"""E07000094""","""E14000811""","""Hampshire""",2022-12-20,"""Mandatory issue (Property to l…",14,56,4,"""Grid Supplied Electricity""",null,null,null,936,26.11,7.24,29.03,32.78,"""No""",null,null,4,"""Air Conditioning""","""Unit B9, Whiteley Shopping Cen…","""Winchester""","""Meon Valley""",null,2023-04-19 22:16:02,355,10090843642,"""Energy Assessor"""
"""5d06cc0176e0055c508bd530c1072d…","""Thames Riviera Hotel""","""162 Bridge Road""","""Maidenhead""","""SL6 8DW""",10003706395,29,"""B""","""Hotels""",2022-10-21,"""E06000040""","""E14000803""",null,2022-11-13,"""Voluntary re-issue (A valid EP…",14,54,5,"""Natural Gas""",null,null,null,1619,83.42,22.73,90.92,48.8,"""Yes""",null,2.0,4,"""Air Conditioning""","""Thames Riviera Hotel, 162 Brid…","""Windsor and Maidenhead""","""Maidenhead""",null,2022-11-14 22:03:47,377,100081276821,"""Energy Assessor"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""d504186fd17376d5db96bdd867589b…","""Barnsley Transport Interchange""","""32-34 Midland Street""","""Barnsley""","""S70 1SE""",10004079437,50,"""B""","""Retail/Financial and Professio…",2023-02-07,"""E08000016""","""E14000541""",null,2023-02-13,"""Mandatory issue (Marketed sale…",9,37,3,"""Grid Supplied Electricity""",null,null,null,167,20.78,3.78,15.18,20.6,"""Yes""",null,2.0,5,"""Air Conditioning""","""Barnsley Transport Interchange…","""Barnsley""","""Barnsley Central""",null,2023-02-13 16:41:40,222,null,null
"""45b0d75db662e